In [1]:
import numpy as np
import glob
from osgeo import gdal, osr
import pyproj
from affine import Affine
from shutil import copyfile

In [2]:
tfw_file = r"D:\FloodChange\GeneralDownload\48157C0305L.tfw"
base_file = r"D:\FloodChange\BaseRaster\BaseTest.tif"

In [3]:
input_epsg = "EPSG:3857"
intermediate_epsg = "EPSG:3857"
output_epsg = "EPSG:32615"
output_esri = "ESRI:102740"

In [4]:
def get_geotransform_from_tfw(tfw_file):
    # READ TFW FILE AND RETURN GDAL GEOTRANSFORM
    with open(tfw_file, 'r') as f:
        tfw = [float(line) for line in f if line is not "\n"]

    geotransform = (tfw[4], tfw[0], tfw[1], tfw[5], tfw[2], tfw[3])
    
    # CHECK LAST OBSERVATION, IF IT'S BIGGER THAN 10e7 IT'S PROLLY STATE PLANE, ELSE UTM
    if np.abs(tfw[5]) > 1e7 :
        outcrs = "ESRI:102740"
    else:
        outcrs = "EPSG:32615"
    return geotransform, outcrs

def affine_to_geotransform(affine_obj):
    """Convert an affine object to a geotransform tuple.
    
    Parameters:
        affine_obj (affine.Affine): The affine object to convert.
    
    Returns:
        tuple: The geotransform tuple (x_origin, x_resolution, x_rotation, y_origin, y_rotation, y_resolution).
    """
    x_origin = affine_obj.c
    y_origin = affine_obj.f
    x_resolution = affine_obj.a
    y_resolution = affine_obj.e
    x_rotation = affine_obj.b
    y_rotation = affine_obj.d
    return (x_origin, x_resolution, x_rotation, y_origin, y_rotation, y_resolution)

def reproject_geotransform_old(geotransform, src_crs, dst_crs):
    src_proj = pyproj.Proj(src_crs)
    dst_proj = pyproj.Proj(dst_crs)
    
    #
    # Create a transformation object to convert between the input and output CRS
    transformer = pyproj.Transformer.from_crs(src_crs, dst_crs)

    # Transform the coordinates of the upper-left and lower-right corners of the input image
    upper_left = transformer.transform(geotransform[0], geotransform[3])
    lower_right = transformer.transform(geotransform[0] + geotransform[1] * geotransform[2],
                              geotransform[3] + geotransform[5] * geotransform[4])
    
    reprojected_geotransform = [upper_left[0], geotransform[1], geotransform[2], upper_left[1], geotransform[4], geotransform[5]]
    
    return reprojected_geotransform

def getDistWithTransform(transformer, x_res,):
    firstpoint  = transformer.transform(0, x_res)
    secondpoint = transformer.transform(0, 0)
    out = np.sqrt((firstpoint[0] - secondpoint[0]) ** 2 + (firstpoint[1] - secondpoint[1]) ** 2)
    out = np.sign(x_res) * out
    return out

def reproject_geotransform(geotransform, original_crs, target_crs):
    """Transform a geotransform to a different CRS using pyproj.

    Parameters:
        geotransform (tuple): The original geotransform to transform.
        original_crs (str): The original CRS of the geotransform.
        target_crs (str): The target CRS for the transformed geotransform.

    Returns:
        tuple: The transformed geotransform in the target CRS (x_origin, x_resolution, x_rotation, y_origin, y_rotation, y_resolution).
    """
    # Create a pyproj transformer object
    transformer = pyproj.Transformer.from_crs(original_crs, target_crs)

    # Transform the geotransform parameters to the target CRS
    x_origin, x_resolution, x_rotation, y_origin, y_rotation, y_resolution = geotransform
    x_origin_transformed, y_origin_transformed, z = transformer.transform(x_origin, y_origin, 0)

    # Transform the scale factors
    # x_res_transformed, y_res_transformed = transformer.transform(0, x_resolution), transformer.transform(0, y_resolution)
    # x_res_transformed, y_res_transformed = x_res_transformed[0], y_res_transformed[0]
    x_res_transformed = getDistWithTransform(transformer, x_resolution,)
    y_res_transformed = getDistWithTransform(transformer, y_resolution,)
    x_rotation        = getDistWithTransform(transformer, x_rotation,)
    y_rotation        = getDistWithTransform(transformer, x_rotation,)

    # Create a new geotransform with the transformed parameters
    transformed_geotransform = (x_origin_transformed, x_res_transformed, x_rotation,
                                y_origin_transformed, y_rotation, y_res_transformed)

    return transformed_geotransform

def get_affine_from_geotransform(geotransform):
    return Affine.from_gdal(*geotransform)

def AffineFromTFW(tfw_file, new_crs, old_crs=None):
    geotransform, detected_crs = get_geotransform_from_tfw(tfw_file)
    if old_crs is None:
        old_crs = detected_crs
    new_geotransform = reproject_geotransform(geotransform, old_crs, new_crs)
    print(new_geotransform)
    affine = get_affine_from_geotransform(new_geotransform)
    return affine
    
def AffineFromTIF(tif_file, new_crs):
    raster_ds = gdal.Open(tif_file)
    geotransform = raster_ds.GetGeoTransform()
    old_crs = raster_ds.GetProjection()
    new_geotransform = reproject_geotransform(geotransform, old_crs, new_crs)
    affine = get_affine_from_geotransform(new_geotransform)
    return affine

def getMatrixFromAffine(affine_transform):
    mt = affine_transform.to_gdal()
    
    matrix = np.array([
        [mt[1], mt[2], mt[0]],
        [mt[4], mt[5], mt[3]],
        [0, 0, 1]
    ])
    
    
    # matrix = np.array(coefficients_tuple).reshape((2, 3))
    # matrix = np.vstack([matrix, [0, 0, 1]])
    return(matrix)

def combineAffine(src_affine, dst_affine):
    return ~src_affine * dst_affine
    
def reprojectAffine(affine_obj, original_crs, target_crs):
    """Transform an affine object to a different CRS using pyproj.
    
    Parameters:
        affine_obj (affine.Affine): The original affine object to transform.
        original_crs (str): The original CRS of the affine object.
        target_crs (str): The target CRS for the transformed affine object.
    
    Returns:
        affine.Affine: The transformed affine object in the target CRS.
    """

    # CONVERT TO GEOTRANSFORM
    print("Inside function")
    geotrans = affine_to_geotransform(affine_obj)
    print(geotrans)
    
    # USE FUNCTIONS TO REPROJECT AND BACK TO AFFINE
    temp = reproject_geotransform(geotrans, original_crs, target_crs)
    print(temp)
    transformed_affine = get_affine_from_geotransform(temp)
    print(transformed_affine)
    
    return transformed_affine

In [5]:
dataloc = r"D:\FloodChange\GeneralDownload\\"
outloc = r"D:\FloodChange\TrainDataset\\"

base_affine=AffineFromTIF(base_file, intermediate_epsg)
files = glob.glob(f"{dataloc}*.tfw")
print(len(files))

for tfw_file in files:
    print(tfw_file)
    # GET FILENAME
    filename = tfw_file.split("\\")[-1]
    filename = filename[:-4]
    
    # CALCULATE AFFINE PARAMETERS
    tfw_affine = AffineFromTFW(tfw_file, intermediate_epsg, old_crs=None)
    combined = combineAffine(base_affine, tfw_affine)
    print(combined)
    
    #combined = reprojectAffine(combined, intermediate_epsg, output_epsg)
    #tfw_affine = reprojectAffine(tfw_affine, intermediate_epsg, output_epsg)
    
    #print(combined)
    #print(tfw_affine)
    
    combined_matrix = getMatrixFromAffine(combined).flatten()[:6]
    tfw_affine_matrix = getMatrixFromAffine(tfw_affine).flatten()[:6]
    
    # SAVE AFFINE TRANSFORMATION
    #text_file = open(f"{outloc}{filename}.affine", "w")
    #matrix = [f"{l2}," for l2 in combined_matrix]
    #print(matrix)
    # n = text_file.write(matrix)
    
    np.save(f"{outloc}{filename}_affine", combined_matrix)
    np.save(f"{outloc}{filename}_affine_abs", tfw_affine_matrix)
    
    # SAVE 
    copyfile(f"{dataloc}{filename}.tif", f"{outloc}{filename}.tif")
    

110
D:\FloodChange\GeneralDownload\48157C0285L.tfw
(-10640662.794482207, 0.6428696024275753, 4.969501396288181e-05, 3456179.79571008, 1.2776895727842184e-05, -0.6428666214711455)
| 0.13, 0.00, 8329.62|
|-0.00, 0.13, 13862.08|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48157C0305L.tfw
(-10633702.202274097, 0.6428151194244893, 1.1232659952066183e-05, 3456184.055121432, 2.885468203531968e-06, -0.6428857911546582)
| 0.13, 0.00, 9721.76|
|-0.00, 0.13, 13861.23|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0170L.tfw
(-10682502.706239533, 0.5101108004515604, -8.085474701720231e-05, 3512025.5032186154, -8.118189635749102e-05, -0.5100628196876704)
| 0.10,-0.00,-38.46|
| 0.00, 0.10, 2693.11|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0195L.tfw
(-10668758.45522495, 0.5100664837574588, -2.6952998569863877e-05, 3512027.514007263, -2.706204834995919e-05, -0.5100412986410361)
| 0.10,-0.00, 2710.42|
| 0.00, 0.10, 2692.71|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDown

D:\FloodChange\GeneralDownload\48201C0460L.tfw
(-10626995.95429318, 0.5100767174411984, -9.129236882642165e-05, 3504014.1768369954, -9.166058886311083e-05, -0.5100703681990261)
| 0.10,-0.00, 11063.03|
| 0.00, 0.10, 4295.35|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0465L.tfw
(-10633952.635931741, 0.510093985543307, -4.693743294554361e-05, 3495979.6252948614, -4.712720788752824e-05, -0.5101336156603085)
| 0.10,-0.00, 9671.67|
| 0.00, 0.10, 5902.24|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0480L.tfw
(-10620036.91452103, 0.7638782169909064, 1.3203520776925094e-05, 3504019.866164449, 1.3257337551517245e-05, -0.7758551835399519)
| 0.15, 0.00, 12454.85|
|-0.00, 0.16, 4294.21|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0485L.tfw
(-10613071.665311316, 0.7629587858316652, -0.00025715000320066704, 3504016.450034408, -0.0002581881004579755, -0.7748660580110899)
| 0.15,-0.00, 13847.92|
| 0.00, 0.15, 4294.90|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDow

D:\FloodChange\GeneralDownload\48201C0765L.tfw
(-10578266.37294175, 0.5100898019975325, -3.066069109310552e-05, 3479960.1429687957, -3.078531941321452e-05, -0.5100138025157134)
| 0.10,-0.00, 20809.07|
| 0.00, 0.10, 9106.08|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0770L.tfw
(-10571302.680733923, 0.7526523337215796, -9.424521027448747e-05, 3479956.959211807, -9.462617638936993e-05, -0.768626630340563)
| 0.15,-0.00, 22201.83|
| 0.00, 0.15, 9106.72|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0770M.tfw
(-10571308.756985515, 0.7650741587047079, 0.0, 3479920.037814352, 0.0, -0.7650741598385693)
| 0.15, 0.00, 22200.61|
| 0.00, 0.15, 9114.11|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0845L.tfw
(-10640894.837638503, 0.510050012993289, 3.6142212831425686e-06, 3463899.8176879133, 3.6283835922457874e-06, -0.5100699334161017)
| 0.10, 0.00, 8283.22|
|-0.00, 0.10, 12318.10|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C0865L.tfw
(-10633930.29138

D:\FloodChange\GeneralDownload\48201C1085L.tfw
(-10585208.975700976, 0.763034354487769, 9.187939653743529e-05, 3455915.613498544, 9.22504490359345e-05, -0.768652860356414)
| 0.15, 0.00, 19420.53|
|-0.00, 0.15, 13914.92|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C1090L.tfw
(-10675658.597002849, 0.507718159244642, -0.0033926002656976557, 3512069.4325579265, -0.003406281056298665, -0.5102829558055173)
| 0.10,-0.00, 1330.37|
| 0.00, 0.10, 2684.32|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C1095L.tfw
(-10585034.928002, 0.5101150647224904, -7.24593301807083e-05, 3447924.9381516366, -7.275248997914844e-05, -0.5100344001797202)
| 0.10,-0.00, 19455.34|
| 0.00, 0.10, 15513.03|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\48201C1105L.tfw
(-10578254.279265504, 0.5101272669670378, -9.034066165467123e-05, 3455927.716673029, -9.070604922953968e-05, -0.5100835504773936)
| 0.10,-0.00, 20811.49|
| 0.00, 0.10, 13912.50|
| 0.00, 0.00, 1.00|
D:\FloodChange\GeneralDownload\

In [6]:
print(np.load(r"D:\FloodChange\TrainDataset\48157C0285L.affine.npy"))

[ 5.00036879e-01  3.86553684e-05  6.93913722e+03 -1.43308306e-05
  5.00020619e-01  1.21431562e+04]
